In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Check TensorFlow Version
print('TF version: {}' .format(tf.__version__))

#Check for GPU utilization
if tf.test.gpu_device_name():
    print(' GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

TF version: 2.2.0
 GPU: /device:GPU:0


In [2]:
# load model

#model path
model_path = './saved_MobileNetV2'

#load it in to tf
model = tf.keras.models.load_model(model_path, compile = True)

In [17]:
#load images from Michael Directory

aerial_folder = '../data/michael20181011a'


aerials = []
labels = []
for img in os.listdir(aerial_folder):
    #pull the name of the image
    labels.append(img)
    #pull the image
    img = os.path.join(aerial_folder, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size=(400,400))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img/255
    img = np.expand_dims(img,axis=0)
    aerials.append(img)
    

In [18]:
#run through model for prediction
aerials = np.vstack(aerials)
wpreds = model.predict(aerials)
print(wpreds)

[[3.9023369e-16]
 [7.1777969e-16]
 [3.1588844e-15]
 ...
 [7.0608121e-21]
 [5.1690423e-14]
 [3.9374968e-21]]


In [20]:
#just an idiot check
np.histogram(wpreds)

(array([2325,   13,    7,   12,    5,    7,    9,   10,    6,  182]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ],
       dtype=float32))

In [26]:
aerial_labels = np.vstack(np.asarray(labels))
print(aerial_labels)

[['S28473773.jpg']
 ['S28469024.jpg']
 ['C28469908.jpg']
 ...
 ['C28471897.jpg']
 ['S28466887.jpg']
 ['C28470552.jpg']]


In [31]:
#make them a dataframe
mpred_df = pd.DataFrame(data=np.hstack((aerial_labels,wpreds)), columns=["image", "prediction"])
mpred_df.head()

,image,prediction
0,S28473773.jpg,3.902337e-16
1,S28469024.jpg,7.177797e-16
2,C28469908.jpg,3.1588844e-15
3,S28470977.jpg,3.5628615e-17
4,P28464980.jpg,7.7584195e-15


In [35]:
#import michael csv
michael_df = pd.read_csv('../data/Michael.csv')

#append/join to catalog df and prediction via the 'image' column in michael.csv save as csv
michael_df = michael_df.join(mpred_df.set_index('image'), on='image')


In [36]:
#save the df as a csv
michael_df.to_csv('../data/preds.csv', index=False)

In [ ]:
#see some examples